In [12]:
from opensearchpy import OpenSearch

index_name = "fed-court-chunks"


# Connect to your cluster
opensearch_client = OpenSearch(
    hosts=[{"host": "opensearch-dev", "port": 9200}],  # adapt to your setup
    http_compress=True
)

import pandas as pd
df = pd.read_parquet('../data/bger-2024-3-text.parquet')


In [10]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

test_cases = [
  {
    "title": "Litige transfrontalier en matière de protection des données",
    "content": "Une entreprise suisse basée à Zurich a transféré des données personnelles de ses clients vers un prestataire basé en Floride. Un client suisse s'est plaint que ses données aient été utilisées à des fins de marketing sans consentement explicite. L’affaire soulève la question de la conformité au droit suisse sur la protection des données et au principe du transfert international."
  },
  {
    "title": "Discrimination dans un marché réglementé",
    "content": "Un ingénieur software demandeur d'emploi de nationalité allemande affirme que sa candidature a été rejetée par une entreprise semi-publique suisse en raison de sa nationalité. Il invoque une discrimination contraire à l’accord sur la libre circulation des personnes et aux dispositions suisses sur l’égalité de traitement."
  },
  {
    "title": "Droit des brevets et concurrence déloyale",
    "content": "Une PME suisse accuse une entreprise concurrente d’avoir copié un procédé breveté dans le domaine de la chimie verte, utilisé dans toute la Suisse. Elle saisit le Tribunal fédéral des brevets pour violation de brevet, demandant l’interdiction de vente et des dommages-intérêts."
  },
  {
    "title": "Responsabilité de l’État en matière de cybersécurité",
    "content": "Un citoyen suisse voit ses données fiscales compromises après une cyberattaque sur un portail cantonal de déclaration en ligne. Il intente une action contre la Confédération, affirmant une responsabilité subsidiaire pour défaut de surveillance ou standards insuffisants de cybersécurité imposés aux cantons."
  },
  {
    "title": "Recours contre une décision de l’OFSP",
    "content": "Un thérapeute en médecine complémentaire attaque une décision de l’Office fédéral de la santé publique (OFSP) lui refusant l'autorisation de facturer via l’assurance de base (LAMal). Il estime que la décision viole son droit de pratiquer et demande au Tribunal administratif fédéral d’annuler la décision."
  },
  {
    "title": "Délit d’initié et marchés financiers",
    "content": "Un cadre supérieur d’une banque zurichoise est poursuivi pour avoir vendu des actions la veille d’une annonce de fusion confidentielle. La FINMA a transmis le dossier au Ministère public de la Confédération pour soupçon de délit d’initié, en application de la Loi sur les bourses (LBVM)."
  },
  {
    "title": "Asile et détention administrative",
    "content": "Un ressortissant érythréen, débouté de sa demande d’asile, fait recours contre sa mise en détention en vue du renvoi, décidée par le SEM. Il soutient que son renvoi est impossible et que sa détention viole l’article 5 CEDH et la jurisprudence fédérale sur la proportionnalité."
  },
  {
    "title": "Affaire douanière et taxation à l'importation",
    "content": "Une société suisse importe des pièces électroniques de Chine via un entrepôt sous douane à Bâle. L’Administration fédérale des douanes lui inflige une amende pour fausse déclaration de valeur. L’entreprise fait recours en affirmant avoir suivi les instructions d’un transitaire autorisé."
  },
  {
    "title": "Liberté religieuse et service civil",
    "content": "Un objecteur de conscience témoigne que le refus de son admission au service civil, malgré ses convictions religieuses pacifistes, viole sa liberté de conscience garantie par la Constitution fédérale. L’affaire est portée devant le Tribunal fédéral."
  },
  {
    "title": "Litige en matière d’accès à l’information",
    "content": "Un journaliste demande à l’OFEV l’accès à un rapport interne sur des risques environnementaux liés à une entreprise de recyclage. Le département refuse en invoquant des raisons de confidentialité industrielle. Le journaliste saisit le Préposé fédéral à la transparence puis fait recours devant le Tribunal administratif fédéral."
  }
]


case_number = 1
query_embedding = model.encode([test_cases[case_number]["content"]])[0]  # shape: (384,)

In [13]:
top_matches = 10
response = opensearch_client.search(
    index=index_name,
    body={
        "size": top_matches,
        "query": {
            "knn": {
                "embedding": {
                    "vector": query_embedding.tolist(),
                    "k": top_matches
                }
            }
        }
    }
)
hit_ids = set()
for hit in sorted(response["hits"]["hits"], key=lambda x: x["_source"]["doc_id"], reverse=True):
    hit_ids.add(hit["_source"]["doc_id"])
    print(hit["_source"]["doc_id"], hit["_source"]["text"], hit["_score"])


114057 Les recourants reprochent ensuite à l'autorité précédente d'avoir justifié, à tort, une inégalité de traitement entre l'Association vaudoise d'aide et de soins à domicile et les organisations de soins à domicile privées, au motif que cette première est une association de droit public autonome dotée de la personnalité juridique, qui est notamment soumise aux contraintes inhérentes à l'accomplissement d'une tâche publique, alors que les secondes sont privées et bénéficient de la liberté économique. En l'occurrence, il ressort du communiqué de presse du 19 février 2021 et de la Directive que la prime n'était pas uniquement destinée au personnel des institutions socio-sanitaires publiques et parapubliques, mais aussi au personnel des institutions privées. Le seul fait d'appartenir au domaine privé ne peut ainsi pas suffire à justifier une différence de traitement entre deux organisations dont le personnel effectue de prime abord les mêmes prestations avec une exposition au Covid-19 

In [14]:


# here extract the documents from df in column "text" for the relevant doc_ids .. .doc_id is the index
# in the dataframe
relevant_docs = df[df.index.isin(hit_ids)]["text"].tolist()

relevant_docs

['Bundesgericht Tribunal fédéral Tribunale federale Tribunal federal {T 1/2} 1C_9/2012 Arrêt du 7 mai 2012 Ire Cour de droit public Composition MM. les Juges Fonjallaz, Président, Aemisegger et Chaix. Greffière: Mme Tornay Schaller. Participants à la procédure Mouvement suisse contre l\'islamisation (MOSCI), représenté par Me Jean-Luc Addor, avocat, recourant, contre Ville de Fribourg, Maison de Ville, place de l\'Hôtel de Ville 3, 1700 Fribourg, Préfecture du district de la Gruyère, Le Château, case postale 192, 1630 Bulle. Objet Usage accru du domaine public (tenue d\'un stand d\'information), recours contre l\'arrêt du Tribunal cantonal du canton de Fribourg, IIe Cour administrative, du 14 novembre 2011. Faits: A. Le Mouvement suisse contre l\'islamisation (ci-après: le MOSCI) est une association à but non lucratif, formée en 2007 aux fins statutaires de "lutter contre l\'islamisation et la désinformation qui s\'y rapporte, informer sur la véritable nature de l\'islam et son caractè

In [15]:
import openai
import os

api_key = os.environ["OPENAI_API_KEY"]

oai_client = openai.OpenAI(api_key=api_key)
# Set the OpenAI API key
question = f"Fournis un avis juridique en te fondant sur les documents joints concernant le cas suivant : {test_cases[case_number]['content']}"

# Combine the documents into a single prompt
context = "\n\n".join([f"Document {i+1}:\n{doc}" for i, doc in enumerate(relevant_docs)])

response = oai_client.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {"role": "system", "content": "You are a legal assistant who answers based only on provided documents."},
        {"role": "user", "content": f"{context}\n\nQuestion: {question}"}
    ],
    temperature=0
)

print(response.choices[0].message.content)


Sur la base des documents fournis, voici un avis juridique concernant la situation d’un ingénieur software demandeur d’emploi de nationalité allemande qui allègue avoir été victime d’une discrimination en raison de sa nationalité par une entreprise semi-publique suisse.

1. **Cadre juridique applicable**

- **Accord sur la libre circulation des personnes (ALCP)** : Cet accord entre la Suisse et l’Union européenne interdit toute discrimination fondée sur la nationalité dans le domaine de l’emploi (cf. Document 3, consid. 7.1.1). La jurisprudence de la Cour de justice de l’Union européenne (CJUE) et du Tribunal fédéral suisse précise que les discriminations directes ou indirectes sont prohibées (Document 3, consid. 7.1).

- **Droit suisse sur l’égalité de traitement** : L’article 8 al. 1 Cst. garantit le principe d’égalité de traitement, interdisant les distinctions arbitraires entre personnes dans des situations comparables (Document 10, consid. 6.2).

2. **Notion de discrimination et c